In [1]:
from feature_process import *
from data_process import *
from pose_cluster import *

In [2]:
add_clusters=['N','Y'] #'N','Y'
feat_types=['bsF','bsH','bscwtF','bscwtH','bsLSTMF','bsLSTMH','bscwtLSTMF','bscwtLSTMH','frame']#'frame','bsF','bsH','bscwtF','bscwtH','bsLSTMF','bsLSTMH','bscwtLSTMF','bscwtLSTMH'
model_types=['svm', 'rf', 'dnn', 'lstm'] #'svm', 'rf', 'dnn', 'lstm'
save_root = r'C:\Users\x\Desktop\final_data/analysis2/'

In [3]:
class Result:
    '''
    save result of 10 iteration of select true test
    '''
    def __init__(self, exp_set):
        self.exp_set = exp_set
        self.train = []
        self.test = []
        self.ttest = []


In [4]:

results = [] # saving result class of each experiment set
exps = []

for feat_type in feat_types:
    # feat type
    if feat_type == 'frame':
        dlc_root = r'..\final_data\mix_landmark5'
        dlc = DataSet(dlc_root)
    else:
        dlc_root = r'..\final_data\mix_landmark7'
        if feat_type[-1]=='H':
            bs_root = r'..\final_data\mix_bsoidfeat'
        else:
            bs_root = r'..\final_data\mix_bsoidfeat2'
        dlc = DataSet(dlc_root, bsoid=bs_root)
    dlc.generate_feature(feat_type=feat_type)

    for add_cluster in add_clusters:
        # add cluster
        if add_cluster == 'Y':
            dlc.pose_cls(sel=['Cap','Capbasal'], sel_num=20, embed=False, k=50, cls_type='km', clf_type='svm')
            classes=4
        else:
            classes=3

        for model_type in model_types:
            if(model_type=='lstm' and feat_type.find('LSTM')==-1):
                continue
            if(feat_type.find('LSTM')!=-1 and model_type!='lstm'):
                continue

            exp = add_cluster+feat_type+model_type
            exps.append(exp)

            # 10 different true test
            res = Result(exp)

            for i in range(10):
                dlc.generate_train_test(split=0.1, motion_del=False, k=i+1)

                # model
                x_train = np.concatenate(dlc.data['x_train'])
                y_train = np.concatenate(dlc.data['y_train'])
                model = Analysis(model_type=model_type, classes=classes)
                model.train(x_train,y_train)
                res.train.append(model.analysis(x_train, y_train))
                x_test = np.concatenate(dlc.data['x_test'])
                y_test = np.concatenate(dlc.data['y_test'])
                res.test.append(model.analysis(x_test, y_test))
                x_val = np.concatenate(dlc.data['x_val'])
                y_val = np.concatenate(dlc.data['y_val'])
                res.ttest.append(model.analysis(x_val, y_val))

            results.append(res)

basal: 41  ,pain: 15  sng: 14  pH7.4: 12
accuracy =  0.9951039469719795
false alarm:  0.002422145328719723
detection rate:  0.990447242726878
accuracy =  0.8077291521676833
false alarm:  0.1252715873114408
detection rate:  0.6779754748737677
accuracy =  0.6470023562265468
false alarm:  0.20412217531661286
detection rate:  0.2948604992657856
accuracy =  0.9947086003875392
false alarm:  0.0021583551062137906
detection rate:  0.9887323943661972
accuracy =  0.8077274824173067
false alarm:  0.12823737712925087
detection rate:  0.6818596390168971
accuracy =  0.5381238667811814
false alarm:  0.32737180382841974
detection rate:  0.24834437086092714
accuracy =  0.9945330637309968
false alarm:  0.002299379167624741
detection rate:  0.9886143931256713
accuracy =  0.8071465721808666
false alarm:  0.12949684680793297
detection rate:  0.6849820336482403
accuracy =  0.6694835680751173
false alarm:  0.20396013976963892
detection rate:  0.3349298665754362
accuracy =  0.9942178940961656
false alarm:  0.